In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import os
print(tf.__version__)

2.8.0


# Nieuwe sectie

In [3]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 40.8 MB/s 
     |████████████████████████████████| 895 kB 59.4 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
os.environ['KAGGLE_USERNAME'] = "brijesh123" # username from the json file
os.environ['KAGGLE_KEY'] = "cav0dd7f2369dw8bc7f71460602f21g1" # key from the json file

In [5]:
!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus

401 - Unauthorized


In [6]:
!unzip entity-annotated-corpus.zip

unzip:  cannot find or open entity-annotated-corpus.zip, entity-annotated-corpus.zip.zip or entity-annotated-corpus.zip.ZIP.


In [7]:
!pip install seqeval

     |████████████████████████████████| 43 kB 882 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=1a10b14c5e7c811efd64eb9ea7a7a13ac7b885569d1e4b7c16d8aa0c7083dd01
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [8]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score

from seqeval.metrics import classification_report,accuracy_score,f1_score
from tqdm import tqdm,trange

In [9]:
df_data = pd.read_csv("ner_dataset.csv",sep=",",encoding="latin1").fillna(method='ffill')
df_data.shape

FileNotFoundError: ignored

In [ ]:
tag_list=df_data.Tag.unique()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test=train_test_split(df_data,test_size=0.20,shuffle=False)

In [ ]:
x_train.shape,x_test.shape

In [ ]:
agg_func = lambda s: [ [w,t] for w,t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]

In [ ]:
x_train_grouped = x_train.groupby("Sentence #").apply(agg_func)
x_test_grouped = x_test.groupby("Sentence #").apply(agg_func)

In [ ]:
x_train_sentences = [[s[0] for s in sent] for sent in x_train_grouped.values]
x_test_sentences = [[s[0] for s in sent] for sent in x_test_grouped.values]


In [ ]:
x_train_tags = [[t[1] for t in tag] for tag in x_train_grouped.values]
x_test_tags = [[t[1] for t in tag] for tag in x_test_grouped.values]

In [ ]:
np.shape(x_train_tags),np.shape(x_test_tags)

In [ ]:
MAX_LENGTH=128
BERT_MODEL="bert-base-multilingual-cased"

BATCH_SIZE=32

pad_token=0,
pad_token_segment_id=0,
sequence_a_segment_id=0,

In [ ]:
from transformers import (
    TF2_WEIGHTS_NAME,
    BertConfig,
    BertTokenizer,
    TFBertForTokenClassification,
    create_optimizer)

In [ ]:
MODEL_CLASSES = {"bert": (BertConfig, TFBertForTokenClassification, BertTokenizer)}

In [ ]:
label_map = {label: i for i, label in enumerate(tag_list)}

In [ ]:
num_labels = len(tag_list) + 1
num_labels

In [ ]:
pad_token_label_id = 0

In [ ]:
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

In [ ]:
config = config_class.from_pretrained(BERT_MODEL,num_labels=num_labels)

In [ ]:
tokenizer = tokenizer_class.from_pretrained(BERT_MODEL,do_lower_case=False)

In [ ]:
model = model_class.from_pretrained(
                BERT_MODEL,
                from_pt=bool(".bin" in BERT_MODEL),
                config=config)

In [ ]:
 model.layers[-1].activation = tf.keras.activations.softmax

In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_seq_length =128

def convert_to_input(sentences,tags):
  input_id_list,attention_mask_list,token_type_id_list=[],[],[]
  label_id_list=[]
  
  for x,y in tqdm(zip(sentences,tags),total=len(tags)):
  
    tokens = []
    label_ids = []

    for word, label in zip(x, y):
      word_tokens = tokenizer.tokenize(word)
      tokens.extend(word_tokens)
      # Use the real label id for the first token of the word, and padding ids for the remaining tokens
      label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

  
    special_tokens_count =  2
    if len(tokens) > max_seq_length - special_tokens_count:
      tokens = tokens[: (max_seq_length - special_tokens_count)]
      label_ids = label_ids[: (max_seq_length - special_tokens_count)]

    label_ids = [pad_token_label_id]+label_ids+[pad_token_label_id]
    inputs = tokenizer.encode_plus(tokens,add_special_tokens=True, max_length=max_seq_length)

    input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
    attention_masks = [1] * len(input_ids)

    attention_mask_list.append(attention_masks)
    input_id_list.append(input_ids)
    token_type_id_list.append(token_type_ids)

    label_id_list.append(label_ids)

  return input_id_list,token_type_id_list,attention_mask_list,label_id_list


In [ ]:
input_ids_train,token_ids_train,attention_masks_train,label_ids_train=convert_to_input(x_train_sentences,x_train_tags)

In [ ]:
input_ids_test,token_ids_test,attention_masks_test,label_ids_test=convert_to_input(x_test_sentences,x_test_tags)

In [ ]:
input_ids_train = pad_sequences(input_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_train = pad_sequences(token_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_train = pad_sequences(attention_masks_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_train = pad_sequences(label_ids_train,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

In [ ]:
np.shape(input_ids_train),np.shape(token_ids_train),np.shape(attention_masks_train),np.shape(label_ids_train),

In [ ]:
input_ids_test = pad_sequences(input_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
token_ids_test = pad_sequences(token_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
attention_masks_test = pad_sequences(attention_masks_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")
label_ids_test = pad_sequences(label_ids_test,maxlen=max_seq_length,dtype="long",truncating="post",padding="post")

In [ ]:
np.shape(input_ids_test),np.shape(token_ids_test),np.shape(attention_masks_test),np.shape(label_ids_test),

In [ ]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

train_ds = tf.data.Dataset.from_tensor_slices((input_ids_train,attention_masks_train,token_ids_train,label_ids_train)).map(example_to_features).shuffle(1000).batch(32).repeat(5)


test_ds=tf.data.Dataset.from_tensor_slices((input_ids_test,attention_masks_test,token_ids_test,label_ids_test)).map(example_to_features).batch(1)


In [ ]:
for x,y in test_ds.take(10):
  print(x,y)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
history = model.fit(train_ds, epochs=3, validation_data=test_ds)